# Milestone 1

## Purpose:
In this notebook, we will attempt to download a data dump containing daily rainfall over NSW, Australia dataset found on [figshare](https://figshare.com/articles/dataset/Daily_rainfall_over_NSW_Australia/14096681). This data dump is comprised of different files contained in a 776.4 MB compressed format. This size of the uncompressed data dump is approximately 6.6 GB. 

In [42]:
# Imports
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

In [43]:
%load_ext rpy2.ipython
%load_ext memory_profiler

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


## Downloading the data

### Creating the API request

We will use the [requests library](https://docs.python-requests.org/en/master/) to generate an http request call to the [figshare API](https://docs.figshare.com/). Specifically, we will make a call to the `/articles` endpoint to retrieve information about the article of interest including the url we need to download the data.

In [4]:
article_id = 14096681
base_url = 'https://api.figshare.com/v2'
headers = {"Content-Type": "application/json"}
endpoint = f'/articles/{article_id}'
output_directory = "rainfall/"

### Making the API call

In [5]:
response = requests.request("GET", base_url+endpoint, headers=headers)
data = json.loads(response.text)
data

{'defined_type_name': 'dataset',
 'embargo_date': None,
 'citation': 'Beuzen, Tomas (2021): Daily rainfall over NSW, Australia. figshare. Dataset. https://doi.org/10.6084/m9.figshare.14096681.v3',
 'url_private_api': 'https://api.figshare.com/v2/account/articles/14096681',
 'embargo_reason': '',
 'references': ['https://www.wcrp-climate.org/wgcm-cmip/wgcm-cmip6',
  'https://pangeo-data.github.io/pangeo-cmip6-cloud/',
  'https://www.longpaddock.qld.gov.au/silo/'],
 'funding_list': [],
 'url_public_api': 'https://api.figshare.com/v2/articles/14096681',
 'id': 14096681,
 'custom_fields': [],
 'size': 814109773,
 'metadata_reason': '',
 'funding': None,
 'figshare_url': 'https://figshare.com/articles/dataset/Daily_rainfall_over_NSW_Australia/14096681',
 'embargo_type': 'file',
 'title': 'Daily rainfall over NSW, Australia',
 'defined_type': 3,
 'embargo_options': [],
 'is_embargoed': False,
 'version': 3,
 'embargo_title': '',
 'url_public_html': 'https://figshare.com/articles/dataset/Dail

The response above contains a `data` json key which is of interest of us. It lists the files corresponding to the article along with their download urls.

In [6]:
files = data["files"]           
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

The data dump is `data.zip` which can be retreived as follows

### Downloading the file of interest

In [7]:
%%time
files_to_dl = ["data.zip"] 
for file in files:
    if file["name"] in files_to_dl:
        if os.path.exists("rainfall/data.zip") != True: # Do not re-download if file exists
            os.makedirs(output_directory, exist_ok=True)
            urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 5.89 s, sys: 6.62 s, total: 12.5 s
Wall time: 8min 12s


#### Observation:
Based on running the above cell, the single act of downloading the data dump to the local machine took around 2mins 9s. This varied greately between our machines and seemed to be largely dependent on network/internet speed.

#### Comparison

Results of the download operation across 4 different machines

|           | CPU               | RAM   | HD         | Operation time |
|-----------|-------------------|-------|------------|----------------|
| Machine 1 | i5-4460 @ 3.20Ghz | 10 GB | 1 TB SSD   |  2 min 9 sec   |
| Machine 2 | i7-7700HQ @2.80Ghz| 16 GB | 1 TB SSD   |  12 min 54 sec |
| Machine 3 |  i5 @1.60Ghz      | 4 GB  | 121 GB SSD |  5 min 9 sec   |
| Machine 4 |  i5 @2Ghz         | 16 GB | 500 GB SSD |  8 min 12 sec  |

### Extracting the data

Here, we attempt to extract the data dump we downloaded `data.zip` into individual uncompressed csv files.

In [8]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 17.6 s, sys: 2.9 s, total: 20.5 s
Wall time: 21.4 s


#### Comparison

Results of the extraction operation across 4 different machines

|           | CPU               | RAM   | HD         | Operation time |
|-----------|-------------------|-------|------------|----------------|
| Machine 1 | i5-4460 @ 3.20Ghz | 10 GB | 1 TB SSD | 19.5 sec   |
| Machine 2 | i7-7700HQ @ 2.80Ghz                  |     16 GB  |   1 TB SSD         |  20.3 sec              |
| Machine 3 | i5 @1.60Ghz      | 4 GB  | 121 GB SSD |  1 min 53 sec   |
| Machine 4 | i5 @2Ghz         | 16 GB      |  500 GB SSD          |  21.4 sec              |

## 4. Combining data CSVs

### Pandas Approach

Here, we attempt to combine the individuals csv files into a single pandas dataframe which we then save to a single csv file called `combined_data.csv`. We create a new column called `model` to be able to identify which dataset each record originally comes from. The names of the models are extracted using regex from the csv file names.

In [9]:
%%time
%memit
files = glob.glob('rainfall/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall(r'[^\/]+(?=\_d)', file)[0]) # use r'[^\\]+(?=\_d)' on Windows machines
                for file in files)
              )
df.to_csv("rainfall/combined_data.csv")

peak memory: 86.14 MiB, increment: 1.29 MiB
CPU times: user 5min 4s, sys: 15.8 s, total: 5min 19s
Wall time: 5min 25s


#### Comparison

Results of the concatenation operation across 4 different machines

|           | CPU               | RAM   | HD         | Operation time |
|-----------|-------------------|-------|------------|----------------|
| Machine 1 | i5-4460 @ 3.20Ghz | 10 GB | 1 TB SSD | 5 min 34 sec  |
| Machine 2 | i7-7700HQ @ 2.80Ghz                  |     16 GB  |   1 TB SSD         |  7 min 38 sec              |
| Machine 3 | i5 @1.60Ghz      | 4 GB  | 121 GB SSD |  Out of memory error   |
| Machine 4 | i5 @2Ghz                  | 16 GB      | 500 GB SSD         |  5 min 25 sec              |

## 5. Load the combined CSV to memory and perform a simple EDA

Check the size of the file

In [10]:
%%sh
du -sh rainfall/combined_data.csv

5.6G	rainfall/combined_data.csv


The size of the combined dataset is 5.6 GB! We attempt to load this entire file into memory using pandas in the next section.

### Pandas approach

In [6]:
%%time
import pandas as pd
df = pd.read_csv("rainfall/combined_data.csv")

CPU times: user 47.4 s, sys: 9.85 s, total: 57.2 s
Wall time: 59.3 s


In [7]:
print(df.shape)

(62513863, 7)


In [8]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


#### Observation:
Attempting to read the above csv file results in a dead kernel in JupyterLab on some of our machines. System Resources Monitor shows a steady increase in RAM usage until 100% after which the jupyterlab notebook crashes as seen in the image below.

![out-of-memory](img/machine1-1.png)

#### Comparison

Results of the loading operation across 4 different machines

|           | CPU               | RAM   | HD         | Operation time |
|-----------|-------------------|-------|------------|----------------|
| Machine 1 | i5-4460 @ 3.20Ghz | 10 GB | 1 TB SSD | OUT OF MEMORY ERROR  |
| Machine 2 | i7-7700HQ @ 2.80Ghz                  |     16 GB  |   1 TB SSD         |  1 min 27s              |
| Machine 3 | i5 @1.60Ghz      | 4 GB  | 121 GB SSD |  Out of memory error   |
| Machine 4 | i5 @2Ghz                  |  16 GB     |  500 GB SSD          |  1 min 6s              |

### Changing dtype of the data

Next, we attempted to change the data types of the numerical columns to reduce the memory required for storing the pandas dataframe `df` and running some simple EDA.

In [9]:
# check the current data types
df.dtypes

time              object
lat_min          float64
lat_max          float64
lon_min          float64
lon_max          float64
rain (mm/day)    float64
model             object
dtype: object

In [10]:
# compare the memeory to store the data for different data types

print(f"Memory usage with float64: {df[['lat_min', 'lat_max','lon_min', 'lon_max', 'rain (mm/day)']].memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df[['lat_min', 'lat_max','lon_min', 'lon_max', 'rain (mm/day)']].astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 2500.55 MB
Memory usage with float32: 1250.28 MB


In [25]:
%%time
%%memit

print(df[['lat_min', 'lat_max','lon_min', 'lon_max', 'rain (mm/day)']].describe())

            lat_min       lat_max       lon_min       lon_max  rain (mm/day)
count  5.924854e+07  6.246784e+07  5.924854e+07  6.246784e+07   5.929456e+07
mean  -3.310482e+01 -3.197757e+01  1.469059e+02  1.482150e+02   1.901827e+00
std    1.963549e+00  1.992067e+00  3.793784e+00  3.809994e+00   5.588275e+00
min   -3.646739e+01 -3.600000e+01  1.406250e+02  1.412500e+02  -3.807373e-12
25%   -3.486911e+01 -3.366221e+01  1.434375e+02  1.450000e+02   3.876672e-06
50%   -3.300000e+01 -3.204188e+01  1.468750e+02  1.481250e+02   6.161705e-02
75%   -3.140170e+01 -3.015707e+01  1.501875e+02  1.513125e+02   1.021314e+00
max   -2.990000e+01 -2.790606e+01  1.537500e+02  1.556250e+02   4.329395e+02
peak memory: 9805.98 MiB, increment: 4392.93 MiB
CPU times: user 14.4 s, sys: 8.52 s, total: 22.9 s
Wall time: 24.1 s


In [26]:
%%time
%%memit
print(df[['lat_min', 'lat_max','lon_min', 'lon_max', 'rain (mm/day)']].astype('float32').describe())

            lat_min       lat_max       lon_min       lon_max  rain (mm/day)
count  5.924854e+07  6.246784e+07  5.924854e+07  6.246784e+07   5.929456e+07
mean  -3.310497e+01 -3.197765e+01  1.469058e+02  1.482150e+02   1.901828e+00
std    1.963549e+00  1.992067e+00  3.793784e+00  3.809994e+00   5.588274e+00
min   -3.646739e+01 -3.600000e+01  1.406250e+02  1.412500e+02  -3.807373e-12
25%   -3.486911e+01 -3.366221e+01  1.434375e+02  1.450000e+02   3.876672e-06
50%   -3.300000e+01 -3.204189e+01  1.468750e+02  1.481250e+02   6.161705e-02
75%   -3.140170e+01 -3.015707e+01  1.501875e+02  1.513125e+02   1.021314e+00
max   -2.990000e+01 -2.790606e+01  1.537500e+02  1.556250e+02   4.329395e+02
peak memory: 7636.74 MiB, increment: 3548.64 MiB
CPU times: user 10.6 s, sys: 4.31 s, total: 14.9 s
Wall time: 15.7 s


#### Observation:

By changing the data types of the numerical columns from 'float64' to 'float32', the memory required for storing the dataframe reduces a lot. Also if we carry out the simple EDA for numerical columns using data type'float32' instead of 'float64', both time and memory usage while performing the EDA decrease as well.

### DASK approach

Due to the limitations faced when loading the dataset using `pandas`, we attempt to use DASK - a python library that allows for parallel computing and works better with large datasets.

In [14]:
import dask.dataframe as dd

In [15]:
%%time
%%memit
# shows time that dask take to merge
ddf = dd.read_csv("rainfall/combined_data.csv",assume_missing=True, dtype={'lon_min': 'object'})
ddf.to_csv("rainfall/combined_data_dask.csv", single_file=True)

peak memory: 5317.79 MiB, increment: 1949.67 MiB
CPU times: user 6min 21s, sys: 25.2 s, total: 6min 46s
Wall time: 6min 2s


In [16]:
%%time
%%memit
## count the number of records for each model
print(ddf["model"].value_counts().compute())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
NorESM2-MM          3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
NorESM2-LM           919800
CanESM5              551880
BCC-ESM1             551880
observed              46020
Name: model, dtype: int64
peak memory: 2833.30 MiB, increment: 1508.89 MiB
CPU times: user 1min 12s, sys: 15.1 s, total: 1min 27s
Wall time: 40.6 s


In [17]:
%%time
%%memit
## calculate the mean and std of rain amount grouping by model
print(ddf.groupby('model').agg({'rain (mm/day)': ['mean', 'std']}).compute().head())

               rain (mm/day)          
                        mean       std
model                                 
ACCESS-CM2          1.787025  5.914188
ACCESS-ESM1-5       2.217501  6.422397
AWI-ESM-1-1-LR      2.026071  5.321889
BCC-CSM2-MR         1.951832  6.200969
CMCC-CM2-HR4        2.279350  5.629965
peak memory: 2037.38 MiB, increment: 1456.37 MiB
CPU times: user 1min 8s, sys: 14.3 s, total: 1min 22s
Wall time: 35.8 s


#### Observation:

DASK's use of parallel computing and optimization for large datasets comes in handy here. The machine that encountered an out of memory error when using pandas was able to successfuly laod the dataset using DASK. This suggests that we can use DASK to potentially work with larger datasets without having to increase or upgrade system resoruces.

### Loading in separate chunks

We also attempted to use pandas's chunksize argument to limit the number of lines that are read into local memory at a time.

In [44]:
%%time
%%memit
import numpy as np
rain_total = 0
num_entries = 0
model_counts = pd.Series(dtype='int32')
for chunk in pd.read_csv("rainfall/combined_data.csv", chunksize=10_000_000):
    model_counts = model_counts.add(chunk["model"].value_counts(),  fill_value=0)

peak memory: 5098.25 MiB, increment: 1662.06 MiB
Wall time: 1min 16s


In [45]:
model_counts

ACCESS-CM2          1932840.0
ACCESS-ESM1-5       1610700.0
AWI-ESM-1-1-LR       966420.0
BCC-CSM2-MR         3035340.0
BCC-ESM1             551880.0
CMCC-CM2-HR4        3541230.0
CMCC-CM2-SR5        3541230.0
CMCC-ESM2           3541230.0
CanESM5              551880.0
EC-Earth3-Veg-LR    3037320.0
FGOALS-f3-L         3219300.0
FGOALS-g3           1287720.0
GFDL-CM4            3219300.0
GFDL-ESM4           3219300.0
INM-CM4-8           1609650.0
INM-CM5-0           1609650.0
KIOST-ESM           1287720.0
MIROC6              2070900.0
MPI-ESM-1-2-HAM      966420.0
MPI-ESM1-2-HR       5154240.0
MPI-ESM1-2-LR        966420.0
MRI-ESM2-0          3037320.0
NESM3                966420.0
NorESM2-LM           919800.0
NorESM2-MM          3541230.0
SAM0-UNICON         3541153.0
TaiESM1             3541230.0
observed              46020.0
dtype: float64

#### Observation:
By using chunksize argument of `read_csv` we are able to work-around the memory limitation we previously experienced with `pandas` loading of large datasets. Here, for example, we were able to calculate the average rain fall per day across all days included in the data dump.

### Loading particular columns

We also attempted to only load some columns that could be the more important for our analysis. 

In [29]:
%%time
%%memit
use_cols = ["time", "lat_min", "lat_max", "lon_min", "lon_max", "rain (mm/day)", "model"]
df = pd.read_csv("rainfall/combined_data.csv", usecols=use_cols)
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-SR5        3541230
CMCC-CM2-HR4        3541230
NorESM2-MM          3541230
CMCC-ESM2           3541230
TaiESM1             3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-ESM4           3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
observed              46020
Name: model, dtype: int64
peak memory: 7082.89 MiB, increment: 1832.58 MiB
CPU times: user 52.1 s, sys: 11.9 s, total: 1min 4s
Wall time: 1min 7s


In [30]:
%%time
%%memit
use_cols = ["time", "model"]
df = pd.read_csv("rainfall/combined_data.csv", usecols=use_cols)
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-SR5        3541230
CMCC-CM2-HR4        3541230
NorESM2-MM          3541230
CMCC-ESM2           3541230
TaiESM1             3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-ESM4           3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
observed              46020
Name: model, dtype: int64
peak memory: 6276.19 MiB, increment: 2917.65 MiB
CPU times: user 39.7 s, sys: 5.54 s, total: 45.3 s
Wall time: 46.8 s


#### Observation:

By only loading some columns that are directly related to the process of interest (in this case counting how many data point of each model), we are able to reduce the memory and time requirement for this process. 

## 6. Perform a simple EDA in R

### Transfer dataframe to R

In order to transfer the dataframe to R we will attempt to use `parquet` - a protable language-agnostic file format for storing dataframes from the Apache.

First we will use the `datasets` module from the `pyarrow` library to create a memory-efficient representation of the combined `csv` file. Arrow represetnation are columnar and memory efficient

In [3]:
import pyarrow.dataset as ds
dataset = ds.dataset("rainfall/combined_data.csv", format="csv")
arrow_table = dataset.to_table()

Then we can write the `Arrow` table to a `parquet` file on disk as follows

In [5]:
import pyarrow.parquet as pq
pq.write_table(arrow_table, 'rainfall/combined_data.parquet')

As seen below, the size of the resulting `parquet` file is much smaller compared to the combined `csv`

In [8]:
%%sh
du -sh rainfall/combined_data.parquet

542M	rainfall/combined_data.parquet


In R, we can utility the `read_parquet` function from `arrow` package to load the file into memory and create an R dataframe. From there, we perform some basic EDA by counting the records we have for each model in the dataset.

In [4]:
## Code adapted from course material for DSCI525 at UBC 2020
%%time
%%R
library(arrow)
start_time <- Sys.time()
r_data <- read_parquet("rainfall/combined_data.parquet")
print(class(r_data))
library(dplyr)
result <- r_data %>% 
    count(model)
end_time <- Sys.time()
print(result)
print(end_time - start_time)


R[write to console]: 
Attaching package: ‘arrow’


R[write to console]: The following object is masked from ‘package:utils’:

    timestamp




[1] "tbl_df"     "tbl"        "data.frame"


R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# A tibble: 28 x 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# … with 18 more rows
Time difference of 7.321429 secs
CPU times: user 8.69 s, sys: 3.45 s, total: 12.1 s
Wall time: 7.75 s


### Rationale for choosing this approach

We decided to use `parquet` file as the mode of exchange between Python and R. `parquet` was choosen for a number of reasons. First, `parquet` files on disk take up less space when compared to conventional file formats such as `csv`. Even compared to the Arrow-based file format `feather`, `parquet` file was approxiamtely 50% smaller than `feather` file for this dataset. (542MB vs 1.1GB). The other reason `parquet` was chosen is performance. Loading and reading of `parquet` files into memory is efficient in both languages, Python and R. As seen in our R test above, loading the dataset into memory and performing some basic EDA took approximately 7.75 seconds. This result can be contrasted with loading times observed when loading the same dataset in `csv` format in Python (fastest time was greater than 60 seconds). 